Welcome to sisl's playground
----------------------------

In this notebook, we get two already ran SIESTA simulations and **just have fun postprocessing with sisl**.

**Get some inspiration from it**. Go wild and break things, this is just a sandboxed playground :)

Prelude: setting things up
-------

### Simulation files

Here we get a zip file with the simulations that we will postprocess and unzip it:

In [ ]:
! wget https://github.com/pfebrer/sisl/releases/download/fake-tag/sisl_playground_files.zip
! unzip sisl_playground_files.zip

After this, you should have two folders:
- `graphene_uc`: Graphene unit cell with 2 atoms.
- `graphene_defect`: Graphene 6x6 supercell where one atom is missing.

You can check the outputs to see that we have the Hamiltonian (`.TSHS`), the density matrix (`.DM`), the electronic density and total potential grids (`.grid.nc`), the main input file (`RUN.fdf`) and the output file (`RUN.out`), amongst other usual SIESTA outputs.

### Installing `sisl`

`sisl` can be simply installed with `pip`. Here we also ask for the extra visualization dependencies.

In [ ]:
! pip install sisl[viz]

### Documentation reference

- Sisl documentation: https://zerothi.github.io/sisl/
- Sisl visualization module documentation: https://zerothi.github.io/sisl/visualization/viz_module/index.html

During this notebook **we hardly discuss any technical detail**, but it's all in the documentation!

Here comes the fun
-------------------

Everything is set up now, we can start coding!

### Importing notebook dependencies

In [ ]:
# Just to define file paths, not related to sisl
from pathlib import Path

# Sisl imports
import sisl
import sisl.viz
from sisl.viz import merge_plots

# To quickly plot the hamiltonian matrix
import plotly.express as px

# For some reason sisl logger interferes with Colab logger.
sisl.nodes.Node.context["log_level"] = "CRITICAL"

### Inspecting the structures

Let's **explore the structures** that we have!

First, we define the path where we can find each of them:

In [ ]:
path_uc = Path("graphene_uc")
path_defect = Path("graphene_defect")

Then, tell sisl where the main fdf file is in each case. `get_sile` creates a `sisl` file for the specific extension. This specific file knows how to read and write things in this format.

In [ ]:
fdf_uc = sisl.get_sile(path_uc / "RUN.fdf")
fdf_defect = sisl.get_sile(path_defect / "RUN.fdf")

As an example, you can directly plot the geometry from an fdf file, because `sisl` knows how to read from an `fdf` file, and it also knows how to plot geometries.

Let's see the unit cell one:

In [ ]:
fdf_uc.plot.geometry(axes="xy")

And then the defect one:

In [ ]:
fdf_defect.plot.geometry(axes="xy")

In fact we can combine both in subplots, courtesy of `sisl.viz.merge_plots`:

In [ ]:
plot_uc = fdf_uc.plot.geometry(axes="xy")
plot_def = fdf_defect.plot.geometry(axes="xy")

merge_plots(plot_uc, plot_def, composite_method="subplots", cols=2)

Now that we have made sure that the structures are what we expected, let's move on!

### Graphene unit cell

First we will warm up by analyzing (not so) boring graphene unit cell case.

#### The Hamiltonian

Of course, `sisl` knows how to read the hamiltonian from the `fdf` file. It will find the `.TSHS` file and read it.

In [ ]:
H_uc = fdf_uc.read_hamiltonian()

Let's see what is the information that it contains:

In [ ]:
H_uc

The hamiltonian is a matrix. But it is a sparse matrix, stored in the **Compressed Sparse Row** (CSR) format.

We can convert it to a numpy array:

In [ ]:
H_matrix = H_uc.tocsr().toarray()
H_matrix

And then plot it as an image:

In [ ]:
# Convert 0s to np.nan so that they are not plotted
# (these are the elements that were outside of the sparsity pattern)
H_matrix[H_matrix == 0] = None
px.imshow(H_matrix)

#### Computing the PDOS

From the Hamiltonian, one can compute the PDOS by diagonalizing it. SIESTA does it, but we can also do it in `sisl`.

In [ ]:
pdos_uc = H_uc.plot.pdos(kgrid=[81, 81, 1], data_Erange=[-10, 10], Erange=[-10, 10], nE=500)

pdos_uc

This is the total DOS, but we can ask for the contribution of each orbital type:

In [ ]:
pdos_uc.split_orbs(on="orbitals", name="$orbitals")

#### Computing bands

The bands are also obtained from the Hamiltonian by diagonalizing it. Therefore we can compute them in `sisl`.

We just need to define a path:

In [ ]:
bands_uc = sisl.BandStructure(
    H_uc,
    points=[[0, 0, 0], [0, 0.5, 0], [1/3, 2/3, 0], [0, 0, 0]],
    divisions=50,
    names=[r'$\Gamma$', r'$M$', r'$K$', r'$\Gamma$']
)

And plot them:

In [ ]:
bands_uc.plot(Erange=[-10,10])

#### Computing fatbands

Fatbands display properties of the bands by varying their width. A usual representation is to show the contribution of groups of orbitals.

We can also easily plot fatbands in `sisl`:

In [ ]:
fat_uc = bands_uc.plot.fatbands(Erange=[-10, 10])

Fatbands need some specification of which contributions you want to show. As we did with the PDOS, we show the contributions of all orbital types:

In [ ]:
fat_uc.split_orbs(on="orbitals", name="$orbitals")

#### Computing wavefunctions

Eigenstates of the hamiltonian can be projected on real space. We can easily do so with `sisl`.

Here, we plot wf 8 at the gamma point:

In [ ]:
H_uc.plot.wavefunction(i=8, axes="xy", grid_prec=0.1, nsc=[3, 3, 1], smooth=True, plot_geom=True, z_range=[0, 3])

But we could compare with other K points. Here is the valence band at $\Gamma$, which then is valence band -1 at $M$:

In [ ]:
wf_gamma = H_uc.plot.wavefunction(i=3, k=(0,0,0), axes="xy", grid_prec=0.1, nsc=[1, 1, 1], smooth=True, plot_geom=True, z_range=[0, 3])# crange=[-0.4, 0.4])
wf_M = H_uc.plot.wavefunction(i=2, k=(0,0.5,0), axes="xy", grid_prec=0.1, nsc=[1, 1, 1], smooth=True, plot_geom=True, z_range=[0, 3])#, crange=[-0.4, 0.4])

merge_plots(wf_gamma, wf_M, composite_method="subplots", cols=2, subplot_titles=["Wavefunction at Gamma", "Wavefunction at M"]).update_layout(showlegend=False)

#### Reading from the output

We can tell `sisl` where the output file is. In this case we need to specify the type of the file (`sisl.io.stdoutSileSiesta`) because there are multiple formats with the extension `.out`.

In [ ]:
out_uc = sisl.get_sile(path_uc / "RUN.out", cls=sisl.io.stdoutSileSiesta)

Then, we can read multiple output properties from it. Let's do forces, energies, and charges:

In [ ]:
out_uc.read_force()

In [ ]:
out_uc.read_energy()

In [ ]:
out_uc.read_charge("hirshfeld")

We can, for example, take the forces and plot them as arrows on the geometry:

In [ ]:
forces = out_uc.read_force()
fdf_uc.plot.geometry(
    axes="xy",
    arrows=[{"data": forces, "scale": 2000, "width": 3, "color": "blue", "name": "Force"}]
)

#### Reading and plotting real space grids.

From the fdf, we can easily read any grid that SIESTA has written to a file. Let's start with the electronic density (rho):

In [ ]:
rho_uc = fdf_uc.read_grid("rho")
rho_uc

This is a sisl `Grid` object, which does many interesting things, but for now we will just plot it:

In [ ]:
rho_uc.plot(axes="xy", plot_geom=True, colorscale="temps")

We can do the same with the total potential:

In [ ]:
vt_uc = fdf_uc.read_grid("totalpotential")
vt_uc.plot(axes="xy", plot_geom=True, colorscale="temps")

Let's now move to the case of defected graphene!

### Graphene with defect

#### Reading from the output

As we did in the case of the unit cell, we will read properties from the output.

This time, along with the forces, we will plot net charges of atoms by coloring them.  

In [ ]:
# Tell sisl where the output file is.
out_defect = sisl.get_sile(path_defect / "RUN.out", cls=sisl.io.stdoutSileSiesta)

In [ ]:
# Read forces and net charges
forces_defect = out_defect.read_force()
charges_defect = out_defect.read_charge("hirshfeld")[: , 0]

In [ ]:
# Plot geometry, using charges as colors and showing forces as arrows
fdf_defect.plot.geometry(
    axes="xy", atoms_style={"color": charges_defect}, atoms_colorscale="RdBu",
    arrows={"data": forces_defect, "color": "green"}
)

#### Comparing rho and VT with the pristine case

We will now compare the electronic density and total potential of the defected structure vs the pristine one.

Let's start with the electronic density. We read the $\rho$ for the defected structure:

In [ ]:
rho_defect = fdf_defect.read_grid("rho")

Then repeat the unit cell $\rho$ 6 times in each direction to make it as big as the defect one:

In [ ]:
rho_sc = rho_uc.tile(6, 0).tile(6, 1)

And plot the difference:

In [ ]:
(rho_defect - rho_sc).plot(axes="xy", plot_geom=True, crange=[-0.03, 0.03])

We can do the same for the total potential:

In [ ]:
vt_defect = fdf_defect.read_grid("totalpotential")
vt_sc = vt_uc.tile(6, 0).tile(6, 1)

(vt_defect - vt_sc).plot(axes="xy", plot_geom=True, crange=[-0.1, 0.1])

#### Computing the PDOS

As we did in the pristine case, we can read the Hamiltonian:

In [ ]:
H_def = fdf_defect.read_hamiltonian()

And compute the PDOS from it. In this case it is much more expensive so we use a bigger gaussian smoothing and less k points.

In [ ]:
pdos_def = H_def.plot.pdos(kgrid=[5, 5, 1], data_Erange=[-10, 10], Erange=[-10,10], nE=500, distribution=sisl.get_distribution("gaussian", 0.25))

And we split on orbital types:

In [ ]:
pdos_def.split_DOS(on="orbitals", name="$orbitals")

Or atoms:

In [ ]:
pdos_def.split_DOS(on="atoms")

We see that very few atoms contribute to the defect state at 0 eV.

We can plot the contribution of each atom in a geometry plot.

**THIS IS ALREADY VERY ADVANCED!!** But it shows the versatility that `sisl` has so that you can do whatever you want.

We get the PDOS data from the plot:

In [ ]:
pdos_data = pdos_def.nodes.pdos_data.get()
pdos_data

Then group the contributions into atoms:

In [ ]:
atom_pdos = pdos_data.sisl.split_orbitals(on="atoms", name="$atoms")
atom_pdos

Select the energies close to the fermi level and integrate over them:

In [ ]:
near_fermi_atom_pdos = atom_pdos.sel(E=slice(-0.5, 0.5), spin="total").sum("E")

And finally plot it:

In [ ]:
fdf_defect.plot.geometry(atoms_style={"color": near_fermi_atom_pdos})

We are done, what now?
----------------------

You can try to use `sisl` in your day to day work, it will probably help you! Navigate through the documentation and explore all the possibilities :)

Thanks for your attention!